<a href="https://colab.research.google.com/github/AliAkbarBadri/topics-for-types/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data

In [3]:
! ls "/content/drive/My Drive/topics-for-types/"

df_topics.csv  df_types.csv  result.csv  sample-acmsmall.pdf


In [4]:
df_topics = pd.read_csv("/content/drive/My Drive/topics-for-types/df_topics.csv")
df_topics.head()

,sentence,اقتصاد,بازی,برنامه‌نویسی,تحصیل,تغذیه,حقوق/قانون,حیوانات,خرید,روابط,روانشناسی,زبان,زیبایی,سرگرمی,سفر,سلامت,فنی,ماشین,مذهبی,ورزش,کار,کامپیوتر/موبایل,کودکان,گیاهان
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [5]:
df_types = pd.read_csv("/content/drive/My Drive/topics-for-types/df_types.csv")
df_types.head()

,sentence,انتخاب,بله/خیر,ترجمه,تعامل,تعریف,توضیح,دلیل,ریاضی,لیست,نظر,واقعیت,پیشنهاد
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,0,0,0,0,1,1,0,0,0,1,0,0
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,0,1,0,0,0,1,0,0,0,1,0,0
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,0,0,0,0,0,1,1,0,0,1,0,0
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,0,0,0,0,0,1,1,0,0,1,0,0
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,0,0,0,0,1,0,0,0,1,1,0,0


# Word2Vec

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 

In [11]:
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import asarray
from numpy import zeros

In [12]:
X = df_types[df_types.columns[0]].values
Y = df_types[df_types.columns[1:]].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [17]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [62]:
vocab_size, X_train.shape, y_train.shape, X_test.shape, y_test.shape

(8082, (1939, 200), (1939, 12), (485, 200), (485, 12))

In [19]:
embeddings_dictionary = dict()

glove_file = open('drive/My Drive/nlp/similarity/embeddings/twitt_wiki_ham_blog.fa.text.100.vec', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [26]:
print(embeddings_dictionary['و'])
print()
print(embedding_matrix[1])

[-2.222517 -0.428454 -1.253941  1.551816 -0.450873  2.649759 -1.07392
  0.406471 -1.016656  0.639931 -0.24969  -0.256207  2.047839 -0.139004
  2.443881 -2.85783  -2.349114 -0.481083 -0.138462  1.582742 -0.23184
 -3.487877 -1.008129 -1.740662  1.301929 -0.497582 -1.897205  0.871072
 -4.947224  0.300258  3.44793   1.904012  0.126853  0.94079  -0.677725
  0.112498  0.333517 -1.833528  0.391443  1.316889 -0.229213 -2.983886
  2.828554  5.009136 -4.385108  1.408675 -3.693358 -0.689139 -0.793945
 -0.98941   0.708806 -0.387987  2.559047 -3.488382 -0.513087 -1.142355
 -0.297812 -1.609838 -2.724513  0.776761 -1.8403    0.693504 -1.936019
 -3.0206    1.102566  2.473104 -1.330484 -0.189547 -2.518092  0.153484
  1.032132  0.840575 -2.495274 -1.533112  2.027918  0.139619  0.710635
  2.268173  2.209062 -0.910259  1.784887  2.117979 -1.443968  1.68699
 -1.255109  0.657385 -0.284593 -3.734695 -0.227355  0.455545 -0.74898
  1.158716 -1.366242 -3.442508 -1.246078  1.401888  0.353083  0.361452
  1.804778

In [89]:
model = tf.keras.Sequential([
    # tf.keras.layers.Input(shape=(maxlen,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim = 100, 
                            weights=[embedding_matrix],
                            trainable=False, name='Embedding_Layer'),
    tf.keras.layers.LSTM(200,
                        # return_sequences=True,
                        # stateful=True,
                        recurrent_initializer='glorot_uniform',
                        name='LSTM_Layer'),
    tf.keras.layers.Dense(12, activation='sigmoid', name='Dense_Layer')
  ], name='LSTM_Model')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())
print()
print(model.layers[0].weights)

Model: "LSTM_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, None, 100)         808200    
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 200)               240800    
_________________________________________________________________
Dense_Layer (Dense)          (None, 12)                2412      
Total params: 1,051,412
Trainable params: 243,212
Non-trainable params: 808,200
_________________________________________________________________
None

[<tf.Variable 'Embedding_Layer/embeddings:0' shape=(8082, 100) dtype=float32, numpy=
array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [-2.222517e+00, -4.284540e-01, -1.253941e+00, ...,  3.614520e-01,
         1.804778e+00, -2.163010e-01],
       [-1.325660e-01, -4.901635e+00, -1.319971e+00, ...,

In [90]:
history = model.fit(X_train, y_train, batch_size=8, epochs=5, verbose=1, validation_split=0.2)

Epoch 1/5
25/25 [==============================] - 1s 42ms/step - loss: 0.4113 - acc: 0.3372 - val_loss: 0.2951 - val_acc: 0.4588
Epoch 2/5
25/25 [==============================] - 1s 25ms/step - loss: 0.2914 - acc: 0.4952 - val_loss: 0.2938 - val_acc: 0.4588
Epoch 3/5
25/25 [==============================] - 1s 25ms/step - loss: 0.2904 - acc: 0.4952 - val_loss: 0.2938 - val_acc: 0.4588
Epoch 4/5
25/25 [==============================] - 1s 25ms/step - loss: 0.2900 - acc: 0.4958 - val_loss: 0.2975 - val_acc: 0.4588
Epoch 5/5
25/25 [==============================] - 1s 25ms/step - loss: 0.2903 - acc: 0.4958 - val_loss: 0.2954 - val_acc: 0.4588


In [91]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

16/16 [==============================] - 0s 8ms/step - loss: 0.2908 - acc: 0.4845
Test Score: 0.2907758355140686
Test Accuracy: 0.4845360815525055


In [ ]:
# model.summary()
# ku.plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True)

In [66]:
# import matplotlib.pyplot as plt

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])

# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train','test'], loc='upper left')
# plt.show()

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])

# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','test'], loc='upper left')
# plt.show()

# LASER

In [3]:
! pip -q install laserembeddings
! python -m laserembeddings download-models

     |████████████████████████████████| 860kB 7.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [4]:
from laserembeddings import Laser
from __future__ import unicode_literals
laser = Laser()

In [27]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [47]:
np.array([1,2,3,4]).shape

(4,)

In [41]:
embedding = laser.embed_sentences(["سلام خوبی آقا؟ چرا؟"], lang='fa')

In [133]:
print(np.asarray(embedding[0]).astype(np.float32).shape)
print(embedding[0].reshape(-1).shape)
# print(embedding[0][0])
# print()
print(embedding[0].tolist())

(1024,)
(1024,)
[0.00013220480468589813, 0.0005632739048451185, -0.0036418449599295855, 0.04224187508225441, 0.018085766583681107, 0.00017751163977663964, 0.002364765154197812, 0.0020738039165735245, -0.00020900974050164223, -0.00010958989150822163, 0.0016585809644311666, -0.00010220512194791809, 0.005281758960336447, 0.0025984228122979403, 0.006219091359525919, 0.00835022609680891, 0.008378100581467152, -0.0004980257945135236, 8.926170266931877e-05, 0.006056675687432289, 0.00531877251341939, -0.0004272315127309412, -0.00020097970264032483, -0.00010922906949417666, 0.0003911723324563354, 0.0008754387963563204, 0.00010981337254634127, 0.000990370404906571, -0.00018827272288035601, 0.00012104511552024633, 0.01741662807762623, 0.008600754663348198, 0.004271915648132563, 0.004119337536394596, 0.001083105686120689, 0.004252106416970491, 0.005414559040218592, 0.004994853865355253, -0.0018967631040140986, -0.0005139067652635276, 0.002989789703860879, 0.005320620723068714, 0.002828591968864202

In [143]:
df_types = pd.read_csv("/content/drive/My Drive/topics-for-types/df_types.csv")

In [144]:
df_types['sentence'] = df_types['sentence'].apply(lambda sent: laser.embed_sentences(sent, lang='fa')[0])

In [145]:
df_types.head()

,sentence,انتخاب,بله/خیر,ترجمه,تعامل,تعریف,توضیح,دلیل,ریاضی,لیست,نظر,واقعیت,پیشنهاد
0,"[0.020942228, -8.114154e-05, -0.0009854112, 0....",0,0,0,0,1,1,0,0,0,1,0,0
1,"[0.00867996, 0.0027223467, -0.00016234924, 0.0...",0,1,0,0,0,1,0,0,0,1,0,0
2,"[0.011158594, 0.018385716, -8.350732e-05, 0.01...",0,0,0,0,0,1,1,0,0,1,0,0
3,"[0.009575151, 0.01364643, 7.700209e-05, 0.0117...",0,0,0,0,0,1,1,0,0,1,0,0
4,"[0.027029106, 0.002408459, 0.00048337242, 0.00...",0,0,0,0,1,0,0,0,1,1,0,0


In [146]:
df_types.shape

(2424, 13)

In [182]:
X = np.array(df_types[df_types.columns[0]])
X = np.vstack(X)

Y = df_types[df_types.columns[1:]].values

# X = np.array(df_types.iloc[:,0:1])
# Y = df_types.iloc[:,1:].values

In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [196]:
len(X_train),len(X_test),len(y_train),len(y_test)

(2181, 243, 2181, 243)

In [197]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=X[0].shape, name='input_layer'),
    tf.keras.layers.Dense(100, activation='sigmoid', name='Dense_Layer'),
    tf.keras.layers.Dense(12, activation='sigmoid', name='Dense_Layer2'),
  ], name='LASER_Model')

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Model: "LASER_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_Layer (Dense)          (None, 100)               102500    
_________________________________________________________________
Dense_Layer2 (Dense)         (None, 12)                1212      
Total params: 103,712
Trainable params: 103,712
Non-trainable params: 0
_________________________________________________________________
None


In [200]:
history = model.fit(X_train, y_train, batch_size=8, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2340 - acc: 0.4834 - val_loss: 0.2448 - val_acc: 0.5927
Epoch 2/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2236 - acc: 0.5550 - val_loss: 0.2338 - val_acc: 0.5767
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.2150 - acc: 0.5906 - val_loss: 0.2299 - val_acc: 0.5881
Epoch 4/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2084 - acc: 0.6175 - val_loss: 0.2241 - val_acc: 0.6133
Epoch 5/20
218/218 [==============================] - 0s 2ms/step - loss: 0.2027 - acc: 0.6370 - val_loss: 0.2187 - val_acc: 0.6705
Epoch 6/20
218/218 [==============================] - 1s 2ms/step - loss: 0.1973 - acc: 0.6611 - val_loss: 0.2154 - val_acc: 0.6934
Epoch 7/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1927 - acc: 0.6686 - val_loss: 0.2116 - val_acc: 0.7071
Epoch 8/20
218/218 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

# BERT

In [ ]:
! pip -q install laserembeddings
! python -m laserembeddings download-models

     |████████████████████████████████| 860kB 7.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
from laserembeddings import Laser
from __future__ import unicode_literals
laser = Laser()

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
np.array([1,2,3,4]).shape

(4,)

In [ ]:
embedding = laser.embed_sentences(["سلام خوبی آقا؟ چرا؟"], lang='fa')

In [ ]:
print(np.asarray(embedding[0]).astype(np.float32).shape)
print(embedding[0].reshape(-1).shape)
# print(embedding[0][0])
# print()
print(embedding[0].tolist())

(1024,)
(1024,)
[0.00013220480468589813, 0.0005632739048451185, -0.0036418449599295855, 0.04224187508225441, 0.018085766583681107, 0.00017751163977663964, 0.002364765154197812, 0.0020738039165735245, -0.00020900974050164223, -0.00010958989150822163, 0.0016585809644311666, -0.00010220512194791809, 0.005281758960336447, 0.0025984228122979403, 0.006219091359525919, 0.00835022609680891, 0.008378100581467152, -0.0004980257945135236, 8.926170266931877e-05, 0.006056675687432289, 0.00531877251341939, -0.0004272315127309412, -0.00020097970264032483, -0.00010922906949417666, 0.0003911723324563354, 0.0008754387963563204, 0.00010981337254634127, 0.000990370404906571, -0.00018827272288035601, 0.00012104511552024633, 0.01741662807762623, 0.008600754663348198, 0.004271915648132563, 0.004119337536394596, 0.001083105686120689, 0.004252106416970491, 0.005414559040218592, 0.004994853865355253, -0.0018967631040140986, -0.0005139067652635276, 0.002989789703860879, 0.005320620723068714, 0.002828591968864202

In [ ]:
df_types = pd.read_csv("/content/drive/My Drive/topics-for-types/df_types.csv")

In [ ]:
df_types['sentence'] = df_types['sentence'].apply(lambda sent: laser.embed_sentences(sent, lang='fa')[0])

In [ ]:
df_types.head()

,sentence,انتخاب,بله/خیر,ترجمه,تعامل,تعریف,توضیح,دلیل,ریاضی,لیست,نظر,واقعیت,پیشنهاد
0,"[0.020942228, -8.114154e-05, -0.0009854112, 0....",0,0,0,0,1,1,0,0,0,1,0,0
1,"[0.00867996, 0.0027223467, -0.00016234924, 0.0...",0,1,0,0,0,1,0,0,0,1,0,0
2,"[0.011158594, 0.018385716, -8.350732e-05, 0.01...",0,0,0,0,0,1,1,0,0,1,0,0
3,"[0.009575151, 0.01364643, 7.700209e-05, 0.0117...",0,0,0,0,0,1,1,0,0,1,0,0
4,"[0.027029106, 0.002408459, 0.00048337242, 0.00...",0,0,0,0,1,0,0,0,1,1,0,0


In [ ]:
df_types.shape

(2424, 13)

In [ ]:
X = np.array(df_types[df_types.columns[0]])
X = np.vstack(X)

Y = df_types[df_types.columns[1:]].values

# X = np.array(df_types.iloc[:,0:1])
# Y = df_types.iloc[:,1:].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

(2181, 243, 2181, 243)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=X[0].shape, name='input_layer'),
    tf.keras.layers.Dense(100, activation='sigmoid', name='Dense_Layer'),
    tf.keras.layers.Dense(12, activation='sigmoid', name='Dense_Layer2'),
  ], name='LASER_Model')

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Model: "LASER_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_Layer (Dense)          (None, 100)               102500    
_________________________________________________________________
Dense_Layer2 (Dense)         (None, 12)                1212      
Total params: 103,712
Trainable params: 103,712
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, batch_size=8, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2340 - acc: 0.4834 - val_loss: 0.2448 - val_acc: 0.5927
Epoch 2/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2236 - acc: 0.5550 - val_loss: 0.2338 - val_acc: 0.5767
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.2150 - acc: 0.5906 - val_loss: 0.2299 - val_acc: 0.5881
Epoch 4/20
218/218 [==============================] - 1s 2ms/step - loss: 0.2084 - acc: 0.6175 - val_loss: 0.2241 - val_acc: 0.6133
Epoch 5/20
218/218 [==============================] - 0s 2ms/step - loss: 0.2027 - acc: 0.6370 - val_loss: 0.2187 - val_acc: 0.6705
Epoch 6/20
218/218 [==============================] - 1s 2ms/step - loss: 0.1973 - acc: 0.6611 - val_loss: 0.2154 - val_acc: 0.6934
Epoch 7/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1927 - acc: 0.6686 - val_loss: 0.2116 - val_acc: 0.7071
Epoch 8/20
218/218 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])